In [1]:
import pandas as pd
import requests
import time
import googlemaps
import concurrent.futures
import io


In [ ]:
#This whole cell just sets up our functions to handle various api calls

# Set up Google Maps API client
def load_api_key(filepath):
    """Reads API key from a text file."""
    with open(filepath, "r") as file:
        return file.read().strip()

API_KEY = load_api_key("../../Google_API_Key.txt")  #Replace with your API key (not provided in git repo)
gmaps = googlemaps.Client(key=API_KEY)

def get_location_details_google(row):
    """Fetch latitude, longitude, full street address, ZIP code, and county from Google Maps API."""
    
    if "District Name" in row:
        query = f"{row['School Name']}, {row['District Name']}, {state}, High School"
    else:
        query = f"{row['School Name']}, {state}, High School"

    
    try:
        geocode_result = gmaps.geocode(query)
        if geocode_result:
            location = geocode_result[0]["geometry"]["location"]
            lat, lon = location["lat"], location["lng"]
            
            
            # Extract address components
            address_components = geocode_result[0].get("address_components", [])
            street_number, street_name, zip_code, county, city = "", "", "N/A", "N/A", "N/A"
            
            for component in address_components:
                types = component["types"]
                if "street_number" in types:
                    street_number = component["long_name"]
                if "route" in types:
                    street_name = component["long_name"]
                if "postal_code" in types:
                    zip_code = component["long_name"]
                if "administrative_area_level_2" in types:
                    county = component["long_name"]
                if "locality" in types:  # This is for city
                    city = component["long_name"]
            
                    
            full_street_address = f"{street_number} {street_name}".strip()
            return pd.Series([lat, lon, full_street_address,  zip_code, county, city])
    
    except Exception as e:
        print(f"Error fetching details for {query}: {e}")
    
    return pd.Series(["N/A", "N/A", "N/A", "N/A", "N/A"])


def fetch_missing_county_names(df, gmaps, state):
    """Fill in missing or 'N/A' county names using Google Maps API."""

    def get_county_only(row):
        if pd.notna(row['county_name']) and row['county_name'].strip().upper() != "N/A":
            return row['county_name'].strip()  # Already exists and is valid

        if pd.isna(row["latitude"]) or pd.isna(row["longitude"]):
            return None

        try:
            result = gmaps.reverse_geocode((row["latitude"], row["longitude"]))
            for component in result[0].get("address_components", []):
                if "administrative_area_level_2" in component["types"]:
                    return component["long_name"].strip()
        except Exception as e:
            print(f"Error for index {row.name}: {e}")
            return None

    df["county_name"] = df.apply(get_county_only, axis=1)
    return df


    df["county_name"] = df.apply(get_county_only, axis=1)
    return df


def get_census_tract(lat, lon):
    """Fetch the census tract for given latitude and longitude using the Census Geocoder API."""
    # url = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={lon}&y={lat}&benchmark=Public_AR_2020&vintage=Census2020_Census2020&layers=10&format=json"
    url = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={lon}&y={lat}&benchmark=Public_AR_Current&vintage=Current_Current&layers=10&format=json"


    try:
        response = requests.get(url, timeout=5)  # Set timeout to avoid hanging requests
        response.raise_for_status()  # Raise an error for HTTP issues
        data = response.json()
        
        geographies = data.get('result', {}).get('geographies', {})
        if 'Census Block Groups' in geographies:
            tract_id = str(geographies['Census Block Groups'][0].get('TRACT', 'Not found'))
            return tract_id.zfill(6)  # Ensure 6-digit format
        return 'No data'
    
    except requests.exceptions.RequestException as e:
        return f"Error: {e}"



def add_missing_census_tracts(df, max_workers=5):
    """Only fetch census tracts for rows where Tract == '000000'."""
    
    # Filter rows with bad tract data
    mask = df['Tract'] == "000000"
    df_missing = df[mask].copy()

    if df_missing.empty:
        print("No missing tracts to update.")
        return df
    
    print(f"Updating {len(df_missing)} missing tracts...")

    # Fetch the good tracts
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        new_tracts = list(executor.map(get_census_tract, df_missing['latitude'], df_missing['longitude']))

    # Update only those rows in the original dataframe
    df.loc[mask, 'Tract'] = new_tracts

    return df



def batch_census_tract(indf):

    #Takes in data frame, indf which has column heads, among others, "Address", "City", "State", and "Zip" and batch looks up census tract info.

    indf["Unique ID"] = indf.index

    df2=indf[["address","city_name","state_name","zip_code","Unique ID"]] #grab just relevant columns

    # Remove rows where 'Address' is NaN or an empty string
    df2 = df2[df2['address'].notna() & (df2['address'] != '')]
    df2 =  df2[df2['city_name'].notna() & (df2['city_name'] != '')]
    df2.columns = ["Street Address","City","State","Zip","Unique ID"]
    df2 = df2[["Unique ID"] + [col for col in df2.columns if col != "unique ID"]] #Unique ID needs to be first column

    csv_buffer = io.StringIO()
    df2.to_csv(csv_buffer, index=False, header=False, quoting=1)  # quoting=1 forces proper quoting
    csv_buffer.seek(0)  # Move to the start of the buffer

    # API Endpoint
    url = "https://geocoding.geo.census.gov/geocoder/geographies/addressbatch"
    # url = "https://geocoding.geo.census.gov/geocoder/geographies/addressbatch?benchmark=Public_AR_2020&vintage=Census2020_Census2020"

    # API Request with in-memory file
    files = {"addressFile": ("addresses.csv", csv_buffer.getvalue())}
    data = {"benchmark": "4", "vintage": "4"}

    response = requests.post(url, files=files, data=data)

    # Read response into a new DataFrame (FIXED!)
    result_buffer = io.StringIO(response.text)

    result_df = pd.read_csv(
    result_buffer,
    header=None,
    quotechar='"',  # This ensures that quoted fields (like addresses and lat/lon) stay intact
    names=[
        "Unique ID", "Street Address", "Match Status", "Match Type", 
        "Standardized Address", "Coordinates", "TigerLine ID", "Side", 
        "State FIPS", "County FIPS", "Tract", "Block"
    ]
    )

    # Split Latitude and Longitude.
    result_df[['Longitude', 'Latitude']] = result_df['Coordinates'].str.split(',', expand=True)
    result_df.drop(columns=['Coordinates'], inplace=True)  # Drop combined colum

    result_df["Tract"] = result_df["Tract"].fillna(0).astype(int)  # Fill NaNs with 0 and then convert
    result_df.loc[result_df["Tract"]==0, "Tract"] = "" 

    #Fill in updated county code when possible
    result_df = result_df.set_index("Unique ID")
    indf["county_code"] = indf["county_code"].fillna(result_df["County FIPS"])
    result_df = result_df.reset_index()

    #Drop Unnec. Columns
    result_df = result_df.drop(columns = ["Match Status", "Match Type", "Standardized Address", "Side","State FIPS","TigerLine ID","Block","County FIPS","Latitude","Longitude"])

    #Merge Results
    indf = indf.merge(result_df,on="Unique ID", how = "left")
    indf = indf.drop(columns = ["Unique ID"])
    indf["Tract"] = indf["Tract"].fillna("")

    #Pad Tract ID to 6 digit format
    indf["Tract"] = indf["Tract"].astype(str).str.zfill(6)

    #return modified input df
    return indf






In [26]:
state = "Massachusetts"  # State
input_ed_csv = "/home/mark/Documents/Erdos/Proj/math_ed_project/data/MA19/MA19_ed_data.csv" # Input education data file
input_census_csv = "/home/mark/Documents/Erdos/Proj/math_ed_project/data/census_data/MA/20/MA20_census_data.csv" #Input census data file
input_county_csv = "~/Documents/Erdos/Proj/math_ed_project/data/county_codes.csv" #Path to County Codes
output_csv = "/home/mark/Documents/Erdos/Proj/math_ed_project/data/MA19/MA19_Combined_Census_Education.csv"  # Output file


def build_csv(input_ed_csv,input_census_csv,input_county_csv,state,output_csv):
    #Takes input_ed_csv, input_census_csv, and input_county_csv, and combines into single csv.
    # *_csv arguments are all filepaths to csv file.
    # state is a string specifying which state this applies to.
   

    #Import and format county code data
    countydf=pd.read_csv(input_county_csv)
    countydf["state_name"] = countydf["state_name"].str.strip()
    countydf["county_name"] = countydf["county_name"].str.strip()
    countydf["county_code"] = countydf["county_code"].astype(str)

    # Read ed_CSV into Pandas DataFrame
    ext = input_ed_csv.split('.')[-1] #Get the file type, since they're not all csv files.
    if ext == "csv":
        df = pd.read_csv(input_ed_csv)
    else:
        df = df = pd.read_excel(input_ed_csv)

    #Can't work with entries with no school name
    df = df.dropna(subset=["School Name"])


    #Read census_CSV into data frame. Make unique ID a string, which ends up being convenient.
    dfcensus = pd.read_csv(input_census_csv)
    dfcensus["unique_id"]=dfcensus["unique_id"].astype(str)

    #Fetch Geo data using google maps API
  # Apply the function to get location details and add the resulting columns to the DataFrame
    df["state_name"] = state
    df[["latitude", "longitude", "address", "zip_code", "county_name", "city_name"]] = df.apply(get_location_details_google, axis=1)

    #For whatever reason google doesn't succeed at grabbing all the county names at first pass.
    fetch_missing_county_names(df,gmaps,state)
    
    #Load County ID data into df
    df = df.merge(countydf, on=["state_name", "county_name"], how="left")


    #First do batch census geoprocessing wherever possible
    df = batch_census_tract(df)


    #only then do individual census geoprocessing, on the stragglers
    df = add_missing_census_tracts(df)

      
    df["unique_id"] = df["county_code"].astype(str) + df["Tract"].astype(str)


    df = df.merge(dfcensus, on="unique_id", how="left")


    return df



df = build_csv(input_ed_csv,input_census_csv,input_county_csv,state,output_csv)
df.head()

/tmp/ipykernel_724776/2417638269.py:173: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result_df.loc[result_df["Tract"]==0, "Tract"] = ""


Updating 129 missing tracts...


,School Name,Tests Taken,Reading / Writing,Writing,Math,state_name_x,latitude,longitude,address,zip_code,...,American Indian and Alaska Native alone (estimate),American Indian and Alaska Native alone (percentage),Asian alone (estimate),Asian alone (percentage),Native Hawaiian and Other Pacific Islander alone (estimate),Native Hawaiian and Other Pacific Islander alone (percentage),Some other race alone (estimate),Some other race alone (percentage),Two or more races alone (estimate),Two or more races alone (percentage)
0,Abby Kelley Foster Charter Public (District) -...,106,536.0,NaN,524.0,Massachusetts,42.306192,-71.802933,10 New Bond Street,01606,...,0.0,0.0,654.0,10.3,0.0,0.0,47.0,0.7,162.0,2.5
1,Abington - Abington High,148,528.0,NaN,525.0,Massachusetts,42.119016,-70.953162,201 Gliniewicz Way,02351,...,0.0,0.0,219.0,3.7,0.0,0.0,0.0,0.0,423.0,7.1
2,Academy Of the Pacific Rim Charter Public (Dis...,1,NaN,NaN,NaN,Massachusetts,42.244480,-71.131898,1 Westinghouse Plaza,02136,...,0.0,0.0,220.0,3.8,0.0,0.0,0.0,0.0,109.0,1.9
3,Acton-Boxborough - Acton-Boxborough Regional High,343,648.0,NaN,677.0,Massachusetts,42.479696,-71.458081,36 Charter Road,01720,...,0.0,0.0,1004.0,26.4,0.0,0.0,0.0,0.0,243.0,6.4
4,Adams-Cheshire - Hoosac Valley High School,73,524.0,NaN,516.0,Massachusetts,42.594559,-73.115036,125 Savoy Road,01225,...,7.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,96.0,3.1


In [28]:
#Finally, check that there weren't too many errors in the merging. If not, write out to csv.
if (df[df["American Indian and Alaska Native alone (estimate)"].isna()].shape)[0] <10:
    df = df.dropna(subset = ["American Indian and Alaska Native alone (estimate)"])
    df.to_csv(output_csv, index=False)
else:
    print("false")
